# Μέρος 2: Εντοπισμός Χωρο-χρονικών Σημείων Ενδιαφέροντος και Εξαγωγή Χαρακτηριστικών σε Βίντεο Ανθρωπίνων Δράσεων

Βλάχος Ιωάννης, 03115013, 10ο, εξάμηνο, ΣΗΜΜΥ, ΕΜΠ

Ακαδημαϊκό Έτός 2019-2020, εαρινό εξάμηνο

## 2.1 Χωρο-χρονικά Σημεία Ενδιαφέροντος

Συμπεριλαμβάνουμε τις απαραίτητες βιβλιοθήκες για την εκτέλεση των πειραμάτων.

In [20]:
from scipy import ndimage
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from scipy.ndimage import convolve, convolve1d
import pickle

Κατασκευάζουμε τον διδιάστατο και τρισδιάστατο γκαουσιανό πυρήνα που θα μας χρειαστούν στη συνέχεια για το φιλτράρισμα της ακολουθίας εικόνων. Ο διδιάστατος κατασκευάζεται με πολλαπλασιασμό μονοδιάστατων πυρήνων ενώ ο τρισδιάστος με ένα ακόμα εξωτερικό γινόμενο το οποίο κανουμε reshape. Η τρίτη διάσταση είναι ο χρόνος και μπορεί να έχει διαφορετική διάσταση από τις άλλες 2.

In [21]:
def gaussian_kernel(std):
    siz = int(2*np.ceil(3*std)+1)
    x = cv2.getGaussianKernel(siz, std)
    return np.multiply(x.T, x)

def gauss3d(std, r):
    sizs = int(2*np.ceil(3*std)+1)
    sizr = int(2*np.ceil(3*r)+1)
    x = cv2.getGaussianKernel(sizs, std).flatten()
    y = cv2.getGaussianKernel(sizr, r).flatten()
    xx = np.outer(x, x)
    return np.outer(xx,y).reshape((x.shape[0], x.shape[0], y.shape[0]))

a = gauss3d(3, 2)
a.shape

(19, 19, 13)

Περιλαμβάνουμε και τις συναρτήσεις της πρώτης εργαστηριακής άσκησης για να ελέγξουμε την λειτουργία του ανιχνευτή γωνίων Harris-Stephens.

In [22]:
def interest_points_visualization(I_, kp_data_, ax=None):
    '''
    Plot keypoints chosen by detectos on image.
    Args:
        I_: Image (if colored, make sure it is in RGB and not BGR).
        kp_data_: Nx3 array, as described in assignment.
        ax: Matplotlib axis to plot on (if None, a new Axes object is created).
    Returns:
        ax: Matplotlib axis where the image was plotted.
    '''
    try:
        I = np.array(I_)
        kp_data = np.array(kp_data_)
    except:
        print('Conversion to numpy arrays failed, check if the inputs (image and keypoints) are in the required format.')
        exit(2)

    try:
        assert(len(I.shape) == 2 or (len(I.shape) == 3 and I.shape[2] == 3))
    except AssertionError as e:
        print('interest_points_visualization: Image must be either a 2D matrix or a 3D matrix with the last dimension having size equal to 3.', file=sys.stderr)
        exit(2)

    try:
        assert(len(kp_data.shape) == 2 and kp_data.shape[1] == 3)
    except AssertionError as e:
        print('interest_points_visualization: kp_data must be a 2D matrix with 3 columns.', file=sys.stderr)
        exit(2)

    if ax is None:
        _, ax = plt.subplots()

    ax.set_aspect('equal')
    ax.imshow(I)
    ax.tick_params(bottom=False, left=False, labelbottom=False, labelleft=False)

    for i in range(len(kp_data)):
        x, y, sigma = kp_data[i]
        circ = Circle((x, y), 3*sigma, edgecolor='g', fill=False, linewidth=2)
        ax.add_patch(circ)

    return ax

def disk_strel(n):
    '''
        Return a structural element, which is a disk of radius n.
    '''
    r = int(np.round(n))
    d = 2*r+1
    x = np.arange(d) - r
    y = np.arange(d) - r
    x, y = np.meshgrid(x,y)
    strel = x**2 + y**2 <= r**2
    return strel.astype(np.uint8)

Διαβάζουμε 200 frames από το βίντεο.

In [23]:
from cv20_lab2_2_utils import read_video
from cv20_lab2_2_utils import show_detection

a = read_video('./running/person01_running_d1_uncomp.avi', 200, 0)
a.shape

(120, 160, 200)

## 2.1.1

Παρακάτω υλοποιείται ο τροποποιημένος Harris Stephens ανιχνευτής γωνιών για αλληλουχία εικόνων.
Για τον υπολογισμό των παραγώγων εκτελούμε convolve1d αλλά κατά μήκος του άξονα κατά τον οποίο θέλουμε να παραγωγίσουμε.
Αφού υπολογίσουμε από αυτές τις παραγώγους τα 9 στοιχεία του πίνακα Μ, εκτελούμε στην τελική συνέλιξη με τον τρισδιάστατο γκαουσιανό εκτελώντας πάλι convolve 1d σε κάθε μία από τις 3 διαστάσεις του πίνακα Μ. Τέλος βρίσκουμε τον πίνακα Η σύμφωνα με την εξίσωση της εκφώνησης της άσκησης. Επιλέγουμε τα στοιχεία με τις μεγαλύτερες τιμές και τα δίνουμε σε κατάλληλη μορφή για να δοθεί ως όρισμα στην show detection.

In [24]:
def HarrisStephens(img, sigma=2.5, r = 1.5, s=2, k=0.007, points=1000):
    sizs = int(2*np.ceil(3*s*sigma)+1)
    sizr = int(2*np.ceil(3*s*r)+1)
    '''
    G = gauss3d(s*sigma, s*r)
    G = np.expand_dims(np.expand_dims(G, axis=-1), -1)
    '''
    ker=np.array([-1,0,1])

    Lx = convolve1d(img, weights=ker, axis=0)
    Ly = convolve1d(img, weights=ker, axis=1)
    Lt = convolve1d(img, weights=ker, axis=2)
    
    Lxx = Lx*Lx
    Lyy = Ly*Ly
    Ltt = Lt*Lt
    
    Lxy = Lx*Ly
    Lxt = Lx*Lt
    Lyt = Ly*Lt
    
    Ms = np.zeros(img.shape+(3,3))
    
    Ms[:,:,:,0,0] = Lxx
    Ms[:,:,:,0,1] = Lxy
    Ms[:,:,:,0,2] = Lxt
    Ms[:,:,:,1,0] = Lxy
    Ms[:,:,:,1,1] = Lyy
    Ms[:,:,:,1,2] = Lyt
    Ms[:,:,:,2,0] = Lxt
    Ms[:,:,:,2,1] = Lyt
    Ms[:,:,:,2,2] = Ltt
    
    #M = convolve(Ms, G)
    M=convolve1d(Ms,weights=cv2.getGaussianKernel(sizs, sigma*s).flatten(), axis=0)
    M=convolve1d(M,weights=cv2.getGaussianKernel(sizs, sigma*s).flatten(), axis=1)
    M=convolve1d(M,weights=cv2.getGaussianKernel(sizr, r*s).flatten(), axis=-1)
    
    H = np.linalg.det(M[:,:,:]) - k*np.trace(M, axis1=-2, axis2=-1)
    ss = np.full((points),sigma)
    p = np.unravel_index(np.argsort(H, axis=None), H.shape)
    detect = np.zeros((4, points))
    detect[0]=p[0][-points:]
    detect[1]=p[1][-points:]
    detect[2]=p[2][-points:]
    detect[3]=ss
    #plt.imshow(H[:,:,10])
    #return H, detect.T
    return detect.T

Για τις 3 κατηγορίες βίντεο καλούμε την παραπάνω συνάρτηση και σώζουμε τα αποτελέσματα.

In [25]:
#H, inp = HarrisStephens(a, sigma=3, r=1.5, s=2, k=0.005, points=500)
inp = HarrisStephens(a, sigma=3, r=1.5, s=2, k=0.005, points=500)

In [26]:
#plt.imshow(H[:,:,10])

In [27]:
show_detection(a, inp, save_path="C:\\Users\\ΓΙΑΝΝΗΣ\\Desktop\\οραση\\lab2\\cv20_lab2_part2_material\\part2-videos\\Harris_running")

In [28]:
a = read_video('./walking/person04_walking_d1_uncomp.avi', 200, 0)
inp = HarrisStephens(a, sigma=3, r=1.5, s=2, k=0.005, points=500)
show_detection(a, inp, save_path="C:\\Users\\ΓΙΑΝΝΗΣ\\Desktop\\οραση\\lab2\\cv20_lab2_part2_material\\part2-videos\\Harris_walking")

In [29]:
a = read_video('./boxing/person01_boxing_d2_uncomp.avi', 200, 0)
inp = HarrisStephens(a, sigma=3, r=1.5, s=2, k=0.005, points=500)
show_detection(a, inp, save_path="C:\\Users\\ΓΙΑΝΝΗΣ\\Desktop\\οραση\\lab2\\cv20_lab2_part2_material\\part2-videos\\Harris_boxing")

In [30]:
b = np.random.rand(2,2,2)
b

array([[[0.53123856, 0.49533034],
        [0.45191158, 0.12644268]],

       [[0.35815544, 0.23486168],
        [0.88613236, 0.16787317]]])

In [31]:
np.unravel_index(np.argsort(b, axis=None), b.shape)

(array([0, 1, 1, 1, 0, 0, 0, 1], dtype=int64),
 array([1, 1, 0, 0, 1, 0, 0, 1], dtype=int64),
 array([1, 1, 1, 0, 0, 1, 0, 0], dtype=int64))

In [32]:
def gauss2d(std):
    sizs = int(2*np.ceil(3*std)+1)
    x = cv2.getGaussianKernel(sizs, std).flatten()
    xx = np.outer(x, x)
    return xx

gauss2d(3).shape

(19, 19)

## 2.1.2 

Ορίζουμε παρακάτω 2 gabor φίλτρα με βάση τους τύπους της εκφώνησης.

In [33]:
def gabor_filter_ev(r):  
    t = np.linspace(-2*r, 2*r, int(4*r+1) )
    v = 4/r
    return np.cos(2*np.pi*v*t)*np.exp(-t**2 / 2 / (r**2))

def gabor_filter_odd(r):  
    t = np.linspace(-2*r, 2*r, int(4*r+1) )
    v = 4/r
    return np.sin(2*np.pi*v*t)*np.exp(-t**2 / 2 / (r**2))

In [34]:
a = read_video('./running/person01_running_d1_uncomp.avi', 20, 0)
a.shape

(120, 160, 20)

In [35]:
def normalize(v):
    for i in range(v.shape[-1]):
        norm = np.linalg.norm(v[:,:,i], 1)
        print(norm)
        v[:,:,i] /= norm
    return v

In [36]:
b=np.random.rand(3,3,3)
normalize(b)

1.719492367962139
1.7266978856684143
2.132085658895803


array([[[0.12724526, 0.52244051, 0.39360759],
        [0.19974574, 0.21661622, 0.15999441],
        [0.57719252, 0.44987167, 0.32633029]],

       [[0.16440793, 0.0918249 , 0.16344721],
        [0.25677499, 0.10930957, 0.37070653],
        [0.24500786, 0.53347513, 0.3200399 ]],

       [[0.47455771, 0.20634593, 0.19527283],
        [0.51682887, 0.53412983, 0.20766272],
        [0.17779962, 0.01665321, 0.35362981]]])

Εφαρμόζουμε τις συνελίξεις και υπολογίζουμε την Η ως την τετραγωνική ενέργεια εξόδου. Τέλος υπολογίζουμε πάλι τον Νχ4 πίνακα όπως και πριν.

In [37]:
def Gabor(img, sigma=2.5, r = 1.5, s=2, k=0.007, points=1000):
    siz = int(2*np.ceil(3*s*sigma)+1)
    smooth = convolve1d(img,cv2.getGaussianKernel(siz, s*sigma).flatten(), axis=0)
    
    smooth = convolve1d(smooth,cv2.getGaussianKernel(siz, s*sigma).flatten(), axis=1)
    asdf=convolve1d(smooth, gabor_filter_ev(r), axis=-1)
    
    ev = convolve1d(smooth, gabor_filter_ev(r), axis=-1)
    odd = convolve1d(smooth, gabor_filter_odd(r), axis=-1)
    
    H = ev**2+odd**2
    
    ss = np.full((points),sigma)
    p = np.unravel_index(np.argsort(H, axis=None), H.shape)
    
    detect = np.zeros((4, points))
    detect[0]=p[0][-points:]
    detect[1]=p[1][-points:]
    detect[2]=p[2][-points:]
    detect[3]=ss
    return detect.T

Τρέχουμε εν συνεχεία τα πειράματα.

In [38]:
a = read_video('./walking/person04_walking_d1_uncomp.avi', 200, 0)
inp = Gabor(a, sigma=3, r=1.5, s=2, k=0.005, points=500)
show_detection(a, inp, save_path="C:\\Users\\ΓΙΑΝΝΗΣ\\Desktop\\οραση\\lab2\\cv20_lab2_part2_material\\part2-videos\\Gabor_walking")

In [39]:
a = read_video('./boxing/person01_boxing_d2_uncomp.avi', 200, 0)
inp = Gabor(a, sigma=3, r=1.5, s=2, k=0.005, points=500)
show_detection(a, inp, save_path="C:\\Users\\ΓΙΑΝΝΗΣ\\Desktop\\οραση\\lab2\\cv20_lab2_part2_material\\part2-videos\\Gabor_boxing")

In [40]:
a = read_video('./running/person01_running_d1_uncomp.avi', 200, 0)
inp = Gabor(a, sigma=3, r=1.5, s=2, k=0.005, points=500)
show_detection(a, inp, save_path="C:\\Users\\ΓΙΑΝΝΗΣ\\Desktop\\οραση\\lab2\\cv20_lab2_part2_material\\part2-videos\\Gabor_running")

In [41]:
c = np.array([[ 1, 2, 3], [-1, 1, 4]])
np.linalg.norm(c)

5.656854249492381

# 2.2 Χωρο-χρονικοί Ιστογραφικοί Περιγραφητές

# 2.2.1

Προσθέτουμε τις συναρτήσεις του πρώτου μέρους για την υλοποιήση των HoG, HOFs. Δίνουμε έτσι ένα επιπλέον όρισμα το οποίο αν είναι True δίνει ΗΟF δηλαδή οπτικό πεδίο, αλλιώς δίνει κατευθυντικές παραγώγους HOGs.

In [42]:
def lk(I1, I2, rho, epsilon, isHOG, d_x0, d_y0):
    x_0,y_0 = np.meshgrid(range(I1.shape[1]),range(I1.shape[0]))
    In_1 = ndimage.map_coordinates(I1,np.array([np.ravel(y_0+d_y0), np.ravel(x_0+d_x0)]))
    In_1 = In_1.reshape((I1.shape))
    
    dIn_1x, dIn_1y = np.gradient(In_1.reshape((I1.shape)))
    #dIn_1x, dIn_1y = np.gradient(I1)
    dIn_1x = dIn_1x.reshape((I1.shape))
    dIn_1y = dIn_1y.reshape((I1.shape))
    
    if isHOG:
        return dIn_1x, dIn_1y
    
    A1 = ndimage.map_coordinates(dIn_1x,np.array([np.ravel(y_0+d_y0), np.ravel(x_0+d_x0)]))
    A2 = ndimage.map_coordinates(dIn_1y,np.array([np.ravel(y_0+d_y0), np.ravel(x_0+d_x0)]))
    
    A1 = A1.reshape((I1.shape))
    A2 = A2.reshape((I1.shape))
    
    A = [A1, A2]
    
    E = I2 - In_1
                    
    G = cv2.getGaussianKernel(rho, (rho-1)/6) 
    
    x = cv2.filter2D(A1**2, -1, G)+epsilon
    y = cv2.filter2D(A1*A2, -1, G)
    z = cv2.filter2D(A1*A2, -1, G)
    w = cv2.filter2D(A2**2, -1, G)+epsilon
    
    a = cv2.filter2D(A1*E, -1, G)
    b = cv2.filter2D(A2*E, -1, G)
    
    det = x*w - y*z
    ux = (w*a - y*b) / det
    uy = (b*x - z*a) / det
    
    dx = d_x0 + ux
    dy = d_y0 + uy
    
    return dx, dy

In [43]:
def find_d(I1, I2, rho, epsilon, isHOG, converge=10):
    x_0,y_0 = np.meshgrid(range(I1.shape[1]),range(I1.shape[0]))
    dx,dy = np.meshgrid(np.zeros(I1.shape[1]),np.zeros(I1.shape[0]))
    #dx,dy = np.meshgrid(np.random.rand(I1.shape[1]),np.random.rand(I1.shape[0]))
    #dx,dy = np.meshgrid(range(I1.shape[1]),range(I1.shape[0]))
    #dx,dy = np.meshgrid(np.ones(I1.shape[1]),np.ones(I1.shape[0]))
    for i in range(converge):
        dx, dy = lk(I1, I2, rho, epsilon, isHOG, dx, dy)
    
    dx_r=cv2.resize(dx, None, fx=0.3, fy=0.3)
    dy_r=cv2.resize(dy, None, fx=0.3, fy=0.3)
    
    x = range(dx_r.shape[1])
    y = range(dx_r.shape[0])
    '''
    plt.quiver(x, y, -dx_r, -dy_r)
    plt.gca().set_aspect('equal')
    plt.gca().invert_yaxis()
    plt.show()
    
    plt.quiver(x_0, y_0, -dx, -dy)
    plt.gca().invert_yaxis()
    plt.show()
    '''
    return dx, dy

In [44]:
a = read_video('./running/person01_running_d1_uncomp.avi', 200, 0)
a.shape

(120, 160, 200)

In [45]:
def step(x):
    if x>0:
        return x
    else:
        return 0

In [46]:
from cv20_lab2_2_utils import orientation_histogram

## 2.2.3

Σε αυτό το σημείο υλοποιείται σύμφωνα με τις οδηγίες του πρώτου εργαστηρίου η orientation histogram. Κατά την εκτέλεση των πειραμάτων δοκιμάζεται και αυτή και η δοσμένη. Σχόλια επεξηγηματικά συνοδεύουν τον κώδικα στο παρακάτω cell.

In [47]:
dx = np.random.rand(33,33)
dy = np.random.rand(33,33)

def orient_hist(dx, dy, nbins, h, w):
    #Υπολογίζουμε τα προσανατολισμένα gradients από τα πεδία δχ, δυ
    #Εφαρμόζουμε συνέλιξη προς τους δύο άξονες και βρίσκουμε τα ix, iy
    g = [-1,0,1]
    Ix = convolve1d(dx, g, axis=0)
    Iy = convolve1d(dy, g, axis=1)
    
    #print('ix.shape', Ix.shape)
    
    #αυτό είναι το magnitude
    M = np.sqrt(Ix**2+Iy**2)
    #print('M.shape', M.shape)
    
    #αυτή είναι η γωνία από τα ιχ, ιυ. υπολογίζεται με numpy, αλλά επειδή επιστρέφει rad 
    #μετατρέπουμε σε μοίρες και κα΄νουμε τις αρνητικές τιμε΄ς θετικές με αντίστοιχες μοίρες που έχουν ίδια τιμή tan.
    Angle = np.arctan2(Iy,Ix)/np.pi*180
    Angle = np.where(Angle>0, Angle, Angle+180)
    
    #print('angle.shape',Angle.shape)
    #Φτιάχνουμε παράθυρα μικρού μεγέθους εδώ 5χ5
    r = 0.25
    cell_size = 4
    total_size = int(cell_size*2*r + cell_size)
    diagrams = []
    
    #μοιράζουμε σε bins τις γωνίες.
    wid = 180 / nbins
    
    #Για κάθε ένα από τα 33χ33 παράθυρα χωρίζουμε τις τιμές του magnitude σε nbins με βάση το width της γωνίας
    #δηλαδή γωνία/wid μας δίνει σε ποιο bin του ιστογράμματος θα μπει η τιμή
    #΄ύστερα συνενώνουμε όλα τα histograms.
    
    for i in range(0, h):         #,cell_size):
        for j in range(0, w):             #,cell_size):
            diag = np.zeros((nbins))
            for k in range(i, min(i+total_size, h)):
                for l in range(j, min(j+total_size, w)):
                    
                    index = int(np.floor(Angle[k,l]/wid))
                    if index==nbins:
                        index=nbins-1
                    
                    diag[index]+=M[k,l]
            diagrams.append(diag)
    
    return np.concatenate( diagrams, axis=0 )


h=orient_hist(dx, dy, 10, 32, 32)
h.shape

(10240,)

Η συνάρτηση αυτή γεμίζει με μηδενικά τα ορίσματα ι1, ι2 ώστε στην τελική να έχουν διαστάσεις he, wi. Είναι πολύ χρήσιμη ώστε να μην βγάζει error η παραπάνω orient_hist.

In [48]:
def pad(i1, i2, he, wi):
    (xdim, ydim) = i1.shape
    arr1 = np.zeros((he, wi))
    arr2 = np.zeros((he, wi))
    
    if xdim<he and ydim<wi:
        arr1[-xdim:, -ydim:]=i1
        arr2[-xdim:, -ydim:]=i2
    elif xdim<he and ydim==wi:
        arr1[-xdim:, :]=i1
        arr2[-xdim:, :]=i2
    elif xdim==he and ydim<wi:
        arr1[:, -ydim:]=i1
        arr2[:, -ydim:]=i2

    return arr1, arr2

# 2.2.2

Η find_d μας δίνει είτε τις κατευθυντικές παραγώγους είτε τα οπτικά πεδία.
Στη συνέχεια δίνουμε αυτά ως input στην orientation histogram είτε στην ατομική υλοποιήση. Για κάθε ένα από τα 500 ανιχνευθέντα σημεία καλούμε την orientation histogram και συνενώνουμε με τις προηγουμενες απεικονίσεις.

In [49]:
def HOF_HOG(a, inp, isHOG, use_given, sigma=4):
    s_window=4*sigma
    width = 2*s_window+1
    height = 2*s_window+1
    total=[]
    for i in range(inp.shape[0]):
        starty = int(step(inp[i][0]-s_window))
        startx = int(step(inp[i][1]-s_window))
        I = a[:,:,int(inp[i][2])]
        I = cv2.normalize(I, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        I1 = I[starty:starty+height, startx:startx+width]
        
        I = a[:,:,int(inp[i][2])+1]
        I = cv2.normalize(I, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        I2 = I[starty:starty+height, startx:startx+width]
        
        if I1.shape!=(height, width):
            #print('fixed')
            I1, I2 = pad(I1, I2, height, width)
        
        dx, dy = find_d(I1, I2, 4, 0.05, isHOG, 5)
        nbins=10
        
        
        if use_given:
            desc = orientation_histogram(dx,dy,nbins,np.array([height, width]))
        else:
            desc = orient_hist(dx,dy,nbins,height,width)
        
        total.append(desc)
        
    return np.vstack(total)

In [50]:
descriptors = HOF_HOG(a, inp, False, True)

Όντως δίνει αποτέλεσμα της διάστασης που περιμέναμε.

In [51]:
descriptors.shape

(500, 10890)

# 2.3: Κατασκευή Bag of Visual Words και χρήση Support Vector Machines για την ταξινόμηση δράσεων


# 2.3.1
Διαβάζουμε όλους τους τίτλους που υπάρχουν στο αρχείο training videos.

In [52]:
with open('traininng_videos.txt') as f:
    content = f.readlines()
train_videos = [x.strip() for x in content] 

Στη συνέχεια διαβάζουμε τους τίτλους όλων των βίντεο και αφαιρούμε τους παραπάνω. Αυτό που προκύπτει είναι το test set.

In [53]:
from os import listdir

allvideos = listdir('boxing')+listdir('walking')+listdir('running')
test_videos = list(set(allvideos)-set(train_videos))
test_videos

['person06_boxing_d1_uncomp.avi',
 'person20_walking_d3_uncomp.avi',
 'person10_running_d2_uncomp.avi',
 'person25_running_d4_uncomp.avi',
 'person15_boxing_d4_uncomp.avi',
 'person07_boxing_d3_uncomp.avi',
 'person24_running_d3_uncomp.avi',
 'person11_walking_d1_uncomp.avi',
 'person11_boxing_d2_uncomp.avi',
 'person07_walking_d4_uncomp.avi',
 'person08_walking_d2_uncomp.avi',
 'person20_running_d1_uncomp.avi']

Για τα παραπάνω σύνολα εργαζόμαστε ως εξής:

Διαβάζουμε τα βίντεο, τα περνάμε από τον ανιχνευτή που θέλουμε, φτιάχνουμε έτσι τα represenations από το HOG/HOF και τα συνενώνουμε όλα.

### Για την τελική μελέτη όλων των πιθανών μεθόδων, περιγραφητών και ανιχνευτών, θα εκτελέσουμε 8 πειράματα που αντιστοιχούν σε Gabor ή Harris, HOG ή HOF και στη δοσμένη υλοποίηση των histograms και μη.

In [94]:
representations_gabor_hog_given = []
representations_gabor_hog_mine = []
representations_gabor_hof_given = []
representations_gabor_hof_mine = []
representations_harris_hog_given = []
representations_harris_hog_mine = []
representations_harris_hof_given = []
representations_harris_hof_mine = []
train_labels =[]
for i in train_videos:
    if 'running' in i:
        file='running'
    elif 'walking' in i:
        file='walking'
    else:
        file='boxing'
    train_labels.append(file)
    a = read_video(os.path.join(file,i), 200, 0)
    inp = HarrisStephens(a[:,:,:200], sigma=4, r=1.5, s=2, k=0.005, points=500)
    descriptors1 = HOF_HOG(a, inp, True, True)
    descriptors2 = HOF_HOG(a, inp, True, False)
    descriptors3 = HOF_HOG(a, inp, False, True)
    descriptors4 = HOF_HOG(a, inp, False, False)
    inp = Gabor(a[:,:,:199], sigma=4, r=1.5, s=2, k=0.005, points=500)
    descriptors5 = HOF_HOG(a, inp, True, True)
    descriptors6 = HOF_HOG(a, inp, True, False)
    descriptors7 = HOF_HOG(a, inp, False, True)
    descriptors8 = HOF_HOG(a, inp, False, False)
    
    representations_harris_hog_given.append(descriptors1)
    representations_harris_hog_mine.append(descriptors2)
    representations_harris_hof_given.append(descriptors3)
    representations_harris_hof_mine.append(descriptors4)
    representations_gabor_hog_given.append(descriptors5)
    representations_gabor_hog_mine.append(descriptors6)
    representations_gabor_hof_given.append(descriptors7)
    representations_gabor_hof_mine.append(descriptors8)
    
train_harris_hog_given = np.stack(representations_harris_hog_given)
train_harris_hog_mine = np.stack(representations_harris_hog_mine)
train_harris_hof_given = np.stack(representations_harris_hof_given)
train_harris_hof_mine = np.stack(representations_harris_hof_mine)
train_gabor_hog_given = np.stack(representations_gabor_hog_given)
train_gabor_hog_mine = np.stack(representations_gabor_hog_mine)
train_gabor_hof_given = np.stack(representations_gabor_hof_given)
train_gabor_hof_mine = np.stack(representations_gabor_hof_mine)

In [95]:
representations_gabor_hog_given = []
representations_gabor_hog_mine = []
representations_gabor_hof_given = []
representations_gabor_hof_mine = []
representations_harris_hog_given = []
representations_harris_hog_mine = []
representations_harris_hof_given = []
representations_harris_hof_mine = []
test_labels=[]
for i in test_videos:
    if 'running' in i:
        file='running'
    elif 'walking' in i:
        file='walking'
    else:
        file='boxing'
    test_labels.append(file)
    a = read_video(os.path.join(file,i), 201, 0)
    inp = HarrisStephens(a[:,:,:200], sigma=4, r=1.5, s=2, k=0.005, points=500)
    descriptors1 = HOF_HOG(a, inp, True, True)
    descriptors2 = HOF_HOG(a, inp, True, False)
    descriptors3 = HOF_HOG(a, inp, False, True)
    descriptors4 = HOF_HOG(a, inp, False, False)
    inp = Gabor(a[:,:,:200], sigma=4, r=1.5, s=2, k=0.005, points=500)
    descriptors5 = HOF_HOG(a, inp, True, True)
    descriptors6 = HOF_HOG(a, inp, True, False)
    descriptors7 = HOF_HOG(a, inp, False, True)
    descriptors8 = HOF_HOG(a, inp, False, False)
    
    representations_harris_hog_given.append(descriptors1)
    representations_harris_hog_mine.append(descriptors2)
    representations_harris_hof_given.append(descriptors3)
    representations_harris_hof_mine.append(descriptors4)
    representations_gabor_hog_given.append(descriptors5)
    representations_gabor_hog_mine.append(descriptors6)
    representations_gabor_hof_given.append(descriptors7)
    representations_gabor_hof_mine.append(descriptors8)
    
test_harris_hog_given = np.stack(representations_harris_hog_given)
test_harris_hog_mine = np.stack(representations_harris_hog_mine)
test_harris_hof_given = np.stack(representations_harris_hof_given)
test_harris_hof_mine = np.stack(representations_harris_hof_mine)
test_gabor_hog_given = np.stack(representations_gabor_hog_given)
test_gabor_hog_mine = np.stack(representations_gabor_hog_mine)
test_gabor_hof_given = np.stack(representations_gabor_hof_given)
test_gabor_hof_mine = np.stack(representations_gabor_hof_mine)

In [ ]:
'''
# Getting back the objects:
with open('objs.pkl') as f:  # Python 3: open(..., 'rb')
    test_harris_hog_given,
            test_harris_hog_mine,
            test_harris_hof_given,
            test_harris_hof_mine,
            test_gabor_hog_given,
            test_gabor_hog_mine,
            test_gabor_hof_given,
            test_gabor_hof_mine, 
            train_harris_hog_given,
            train_harris_hog_mine,
            train_harris_hof_given,
            train_harris_hof_mine,
            train_gabor_hog_given,
            train_gabor_hog_mine,
            train_gabor_hof_given,
            train_gabor_hof_mine = pickle.load(f)
'''

## 2.3.2

Με χρήση της δοσμένης bag of words φέρνουμε όλες τις αναπαραστάσεις σε κοινή διάσταση.

In [96]:
from cv20_lab2_2_utils import bag_of_words

In [97]:
D=500
bow_train, bow_test = bag_of_words(train_harris_hog_given, test_harris_hog_given, num_centers=D)

## 2.3.3
Καλούμε στη συνέχεια τη συνάρτηση train και βλέπουμε τα αποτελέσματα.

In [98]:
from cv20_lab2_2_utils import svm_train_test

accuracy, pred = svm_train_test(bow_train, train_labels, bow_test, test_labels)

In [99]:
print(accuracy, pred)

0.3333333333333333 ['walking' 'boxing' 'running' 'running' 'running' 'running' 'running'
 'running' 'running' 'running' 'boxing' 'running']


Για την τελική μελέτη όλων των πιθανών μεθόδων, περιγραφητών και ανιχνευτών, θα εκτελέσουμε 8 πειράματα που αντιστοιχούν σε 
Gabor ή Harris, HOG ή HOF και στη δοσμένη υλοποίηση των histograms και μη.

In [100]:
bow_train, bow_test = bag_of_words(train_harris_hog_given, test_harris_hog_given, num_centers=D)
accuracy, pred = svm_train_test(bow_train, train_labels, bow_test, test_labels)
print('harris_hog_given')
print(accuracy, pred)

harris_hog_given
0.3333333333333333 ['walking' 'boxing' 'running' 'running' 'running' 'running' 'running'
 'running' 'running' 'running' 'boxing' 'running']


In [101]:
bow_train, bow_test = bag_of_words(train_harris_hog_mine, test_harris_hog_mine, num_centers=D)
accuracy, pred = svm_train_test(bow_train, train_labels, bow_test, test_labels)
print('harris_hog_mine')
print(accuracy, pred)

harris_hog_mine
0.3333333333333333 ['walking' 'walking' 'boxing' 'boxing' 'boxing' 'boxing' 'running'
 'running' 'boxing' 'boxing' 'walking' 'running']


In [102]:
bow_train, bow_test = bag_of_words(train_harris_hof_given, test_harris_hof_given, num_centers=D)
accuracy, pred = svm_train_test(bow_train, train_labels, bow_test, test_labels)
print('harris_hof_given')
print(accuracy, pred)

harris_hof_given
0.25 ['walking' 'boxing' 'running' 'boxing' 'running' 'running' 'running'
 'running' 'running' 'running' 'boxing' 'running']


In [103]:
bow_train, bow_test = bag_of_words(train_harris_hof_mine, test_harris_hof_mine, num_centers=D)
accuracy, pred = svm_train_test(bow_train, train_labels, bow_test, test_labels)
print('harris_hof_mine')
print(accuracy, pred)

harris_hof_mine
0.4166666666666667 ['walking' 'boxing' 'boxing' 'boxing' 'walking' 'walking' 'walking'
 'running' 'boxing' 'boxing' 'walking' 'walking']


In [104]:
bow_train, bow_test = bag_of_words(train_gabor_hog_given, test_gabor_hog_given, num_centers=D)
accuracy, pred = svm_train_test(bow_train, train_labels, bow_test, test_labels)
print('gabor_hog_given')
print(accuracy, pred)

gabor_hog_given
0.6666666666666666 ['running' 'boxing' 'running' 'running' 'boxing' 'running' 'running'
 'running' 'boxing' 'running' 'boxing' 'boxing']


In [105]:
bow_train, bow_test = bag_of_words(train_gabor_hog_mine, test_gabor_hog_mine, num_centers=D)
accuracy, pred = svm_train_test(bow_train, train_labels, bow_test, test_labels)
print('gabor_hog_mine')
print(accuracy, pred)

gabor_hog_mine
0.6666666666666666 ['running' 'boxing' 'running' 'running' 'boxing' 'running' 'running'
 'running' 'running' 'running' 'walking' 'boxing']


In [106]:
bow_train, bow_test = bag_of_words(train_gabor_hof_given, test_gabor_hof_given, num_centers=D)
accuracy, pred = svm_train_test(bow_train, train_labels, bow_test, test_labels)
print('gabor_hof_given')
print(accuracy, pred)

gabor_hof_given
0.6666666666666666 ['running' 'boxing' 'running' 'boxing' 'boxing' 'running' 'walking'
 'running' 'boxing' 'running' 'running' 'boxing']


In [107]:
bow_train, bow_test = bag_of_words(train_gabor_hof_mine, test_gabor_hof_mine, num_centers=D)
accuracy, pred = svm_train_test(bow_train, train_labels, bow_test, test_labels)
print('gabor_hof_mine')
print(accuracy, pred)

gabor_hof_mine
0.6666666666666666 ['running' 'boxing' 'walking' 'walking' 'walking' 'running' 'boxing'
 'running' 'boxing' 'running' 'walking' 'boxing']


Δοκιμάζουμε τέλος τα gabor φίλτρα με λιγότερα centroids και παρατηρούμε τα αποτελέσματα.

In [108]:
D=200

In [109]:
bow_train, bow_test = bag_of_words(train_gabor_hog_given, test_gabor_hog_given, num_centers=D)
accuracy, pred = svm_train_test(bow_train, train_labels, bow_test, test_labels)
print('gabor_hog_given')
print(accuracy, pred)

gabor_hog_given
0.75 ['running' 'boxing' 'running' 'boxing' 'boxing' 'running' 'walking'
 'running' 'boxing' 'running' 'walking' 'boxing']


In [110]:
bow_train, bow_test = bag_of_words(train_gabor_hof_given, test_gabor_hof_given, num_centers=D)
accuracy, pred = svm_train_test(bow_train, train_labels, bow_test, test_labels)
print('gabor_hof_given')
print(accuracy, pred)

gabor_hof_given
0.9166666666666666 ['running' 'boxing' 'walking' 'boxing' 'boxing' 'running' 'walking'
 'running' 'boxing' 'walking' 'walking' 'boxing']


Παρατηρούμε ότι έχουμε πολυ καλύτερα αποτελέσματα. Για να δοκιμάσουμε την αποτελεσματικότητα των μοντέλων, μειώνουμε κι άλλο το D και εκτελούμε όλα τα Gabor μαζί με το καλύτερο από τα παραπάνω Harris.

In [111]:
D = 100

In [112]:
bow_train, bow_test = bag_of_words(train_harris_hof_mine, test_harris_hof_mine, num_centers=D)
accuracy, pred = svm_train_test(bow_train, train_labels, bow_test, test_labels)
print('harris_hof_mine')
print(accuracy, pred)

harris_hof_mine
0.5 ['running' 'boxing' 'running' 'running' 'boxing' 'boxing' 'boxing'
 'running' 'boxing' 'running' 'boxing' 'walking']


In [113]:
bow_train, bow_test = bag_of_words(train_gabor_hog_given, test_gabor_hog_given, num_centers=D)
accuracy, pred = svm_train_test(bow_train, train_labels, bow_test, test_labels)
print('gabor_hog_given')
print(accuracy, pred)

gabor_hog_given
0.8333333333333334 ['running' 'boxing' 'running' 'boxing' 'boxing' 'running' 'walking'
 'running' 'boxing' 'walking' 'walking' 'boxing']


In [114]:
bow_train, bow_test = bag_of_words(train_gabor_hog_mine, test_gabor_hog_mine, num_centers=D)
accuracy, pred = svm_train_test(bow_train, train_labels, bow_test, test_labels)
print('gabor_hog_mine')
print(accuracy, pred)

gabor_hog_mine
0.6666666666666666 ['running' 'boxing' 'running' 'running' 'boxing' 'running' 'running'
 'running' 'running' 'walking' 'walking' 'walking']


In [115]:
bow_train, bow_test = bag_of_words(train_gabor_hof_given, test_gabor_hof_given, num_centers=D)
accuracy, pred = svm_train_test(bow_train, train_labels, bow_test, test_labels)
print('gabor_hof_given')
print(accuracy, pred)

gabor_hof_given
0.9166666666666666 ['running' 'boxing' 'walking' 'boxing' 'boxing' 'running' 'walking'
 'running' 'boxing' 'walking' 'walking' 'boxing']


In [116]:
bow_train, bow_test = bag_of_words(train_gabor_hof_mine, test_gabor_hof_mine, num_centers=D)
accuracy, pred = svm_train_test(bow_train, train_labels, bow_test, test_labels)
print('gabor_hof_mine')
print(accuracy, pred)

gabor_hof_mine
0.8333333333333334 ['running' 'boxing' 'walking' 'walking' 'boxing' 'running' 'boxing'
 'running' 'boxing' 'walking' 'walking' 'boxing']


In [118]:
import pickle

# obj0, obj1, obj2 are created here...

# Saving the objects:
with open('objs.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([test_harris_hog_given,
            test_harris_hog_mine,
            test_harris_hof_given,
            test_harris_hof_mine,
            test_gabor_hog_given,
            test_gabor_hog_mine,
            test_gabor_hof_given,
            test_gabor_hof_mine, 
            train_harris_hog_given,
            train_harris_hog_mine,
            train_harris_hof_given,
            train_harris_hof_mine,
            train_gabor_hog_given,
            train_gabor_hog_mine,
            train_gabor_hof_given,
            train_gabor_hof_mine], f)

MemoryError: 

Επιλέγοντας τον καλύτερο συνδυασμό δοκιμάζουμε και άλλα splits των βίντεο

In [68]:
a = {'s':1}
print(a)

{'s': 1}


In [69]:
allvideos = listdir('boxing')+listdir('walking')+listdir('running')

print(len(allvideos))
#test_videos = list(set(allvideos)-set(train_videos))

48


In [70]:
import random 

train_videos = random.sample(allvideos, 36)
test_videos = list(set(allvideos)-set(train_videos))

In [71]:
train_videos

['person06_running_d2_uncomp.avi',
 'person25_walking_d4_uncomp.avi',
 'person03_running_d3_uncomp.avi',
 'person10_running_d2_uncomp.avi',
 'person02_running_d2_uncomp.avi',
 'person11_boxing_d2_uncomp.avi',
 'person24_running_d3_uncomp.avi',
 'person01_running_d1_uncomp.avi',
 'person22_boxing_d2_uncomp.avi',
 'person04_boxing_d1_uncomp.avi',
 'person21_boxing_d1_uncomp.avi',
 'person03_boxing_d4_uncomp.avi',
 'person20_walking_d3_uncomp.avi',
 'person02_boxing_d3_uncomp.avi',
 'person18_walking_d4_uncomp.avi',
 'person07_boxing_d3_uncomp.avi',
 'person12_walking_d2_uncomp.avi',
 'person05_boxing_d2_uncomp.avi',
 'person16_walking_d2_uncomp.avi',
 'person01_boxing_d2_uncomp.avi',
 'person15_boxing_d4_uncomp.avi',
 'person14_walking_d4_uncomp.avi',
 'person05_running_d1_uncomp.avi',
 'person24_boxing_d4_uncomp.avi',
 'person07_running_d3_uncomp.avi',
 'person06_boxing_d1_uncomp.avi',
 'person11_walking_d1_uncomp.avi',
 'person04_walking_d1_uncomp.avi',
 'person17_walking_d3_uncomp.avi

In [72]:
representations_gabor_hof_given = []
train_labels =[]
balance = {'running':0, 'walking':0, 'boxing':0}
for i in train_videos:
    if 'running' in i:
        file='running'
    elif 'walking' in i:
        file='walking'
    else:
        file='boxing'
    balance[file]+=1
    train_labels.append(file)
    a = read_video(os.path.join(file,i), 200, 0)
    inp = Gabor(a[:,:,:199], sigma=4, r=1.5, s=2, k=0.005, points=500)
    descriptors7 = HOF_HOG(a, inp, False, True)
    representations_gabor_hof_given.append(descriptors7)
print(balance)

{'running': 10, 'walking': 12, 'boxing': 14}


In [73]:
train_gabor_hof_given = np.stack(representations_gabor_hof_given)

In [74]:
representations_gabor_hof_given = []
test_labels=[]
balance = {'running':0, 'walking':0, 'boxing':0}
for i in test_videos:
    if 'running' in i:
        file='running'
    elif 'walking' in i:
        file='walking'
    else:
        file='boxing'
    balance[file]+=1
    test_labels.append(file)
    a = read_video(os.path.join(file,i), 201, 0)
    inp = Gabor(a[:,:,:200], sigma=4, r=1.5, s=2, k=0.005, points=500)
    descriptors7 = HOF_HOG(a, inp, False, True)
    
    representations_gabor_hof_given.append(descriptors7)
    
test_gabor_hof_given = np.stack(representations_gabor_hof_given)
print(balance)

{'running': 6, 'walking': 4, 'boxing': 2}


In [75]:
from cv20_lab2_2_utils import bag_of_words, svm_train_test
D=100
bow_train, bow_test = bag_of_words(train_gabor_hof_given, test_gabor_hof_given, num_centers=D)
accuracy, pred = svm_train_test(bow_train, train_labels, bow_test, test_labels)
print('gabor_hof_given')
print(accuracy, pred)

gabor_hof_given
0.9166666666666666 ['walking' 'running' 'running' 'walking' 'running' 'walking' 'boxing'
 'running' 'running' 'boxing' 'walking' 'walking']
